<a href="https://colab.research.google.com/github/ssheggrud/Mod_20_Project/blob/05_Pooja/Mod_20_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz

!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("FinalProject").getOrCreate()

In [3]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/2005_expenditure.csv"
spark.sparkContext.addFile(url)
df2 = spark.read.csv(SparkFiles.get("2005_expenditure.csv"), sep=",", header=True)
df2.show()

+--------+--------+------+--------+------------+---------+------+---------+------+--------+------+----------+--------+-------------------+-------------------+--------------------+------+-------+-----+--------------------+---------+--------------+-----+-----+----------+--------+---------+----------------+----------------+--------+------+-------+
|ELECTION|OFFICECD|CANDID|CANCLASS|    CANDLAST|CANDFIRST|CANDMI|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|           INV_DATE|               DATE|                NAME|C_CODE|ORG_IND|STRNO|             STRNAME|APARTMENT|          CITY|STATE|  ZIP|PAY_METHOD|    AMNT|PURPOSECD|         PURPOSE|         EXPLAIN|EXEMPTCD|RR_IND|SEG_IND|
+--------+--------+------+--------+------------+---------+------+---------+------+--------+------+----------+--------+-------------------+-------------------+--------------------+------+-------+-----+--------------------+---------+--------------+-----+-----+----------+--------+---------+----------------+-

In [15]:
df3 = df2.filter((df2.OFFICECD=="1") & (df2.CANCLASS=="P"))

In [16]:
df3.show()

+--------+--------+------+--------+--------+----------+------+---------+------+--------+------+----------+--------+------------------+-------------------+--------------------+------+-------+-----+--------------------+---------+--------+-----+-----+----------+--------+---------+----------------+--------------------+--------+------+-------+
|ELECTION|OFFICECD|CANDID|CANCLASS|CANDLAST| CANDFIRST|CANDMI|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|          INV_DATE|               DATE|                NAME|C_CODE|ORG_IND|STRNO|             STRNAME|APARTMENT|    CITY|STATE|  ZIP|PAY_METHOD|    AMNT|PURPOSECD|         PURPOSE|             EXPLAIN|EXEMPTCD|RR_IND|SEG_IND|
+--------+--------+------+--------+--------+----------+------+---------+------+--------+------+----------+--------+------------------+-------------------+--------------------+------+-------+-----+--------------------+---------+--------+-----+-----+----------+--------+---------+----------------+--------------------+--

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://databootcamp-final-05.s3.amazonaws.com/Resources/Individual_Contributions_2017_(X).csv"
spark.sparkContext.addFile(url)
df3 = spark.read.csv(SparkFiles.get("Individual_Contributions_2017_(X).csv"), sep=",", header=True)

df3.show()

+--------+--------+-------+--------+-------------------+---------+------+--------+------+----------+--------+----------+----------+------------------+------+-----+-------+---------+---------+------------------+-----+-----+------------------+--------------------+--------+------------------+-------------+--------+-----+---------+--------+----------+--------+----------------+--------+--------+--------+---------------+-----+------+----------+----------+----------+----------+--------+---------+---------+--------+---------+------+-------+----------+
|ELECTION|OFFICECD|RECIPID|CANCLASS|          RECIPNAME|COMMITTEE|FILING|SCHEDULE|PAGENO|SEQUENCENO|   REFNO|      DATE|REFUNDDATE|              NAME|C_CODE|STRNO|STRNAME|APARTMENT|BOROUGHCD|              CITY|STATE|  ZIP|        OCCUPATION|             EMPNAME|EMPSTRNO|        EMPSTRNAME|      EMPCITY|EMPSTATE| AMNT|MATCHAMNT|PREVAMNT|PAY_METHOD|INTERMNO|      INTERMNAME|INTSTRNO|INTSTRNM|INTAPTNO|        INTCITY|INTST|INTZIP|INTEMPNAME|INTEMPST